In [34]:
import ipywidgets as widgets
from ruamel.yaml import YAML
from dataclasses import dataclass
from IPython.display import display
import os
import sys
from uuid import uuid4
from functools import partial, partialmethod
#print(widgets.__version__)
import collections
from ipyfilechooser import FileChooser

def partialclass(cls, *args, **kwds):

    class NewCls(cls):
        __init__ = partialmethod(cls.__init__, *args, **kwds)

    return NewCls


if __name__ == '__main__':
    Config = partialclass(collections.defaultdict, list)
    assert isinstance(Config(), Config)
yaml = YAML()

In [14]:
base_dir = "/home/fxlyonnet/repos/shape-ce/examples/test"
#config_tpl_path = f"{base_dir}/config/config.tpl.yaml"
#config_path = f"{base_dir}/config/config.didytest.yaml"
config_path = os.getenv("CONFIG_FILE_PATH", f"{base_dir}/config/test.yaml")
#print(config_path)
#print(os.getcwd())
#with open(config_tpl_path, "r") as config_tpl_fd:
#    config_tpl = yaml.safe_load(config_tpl_fd)

config_path = f"{base_dir}/config/test.yaml"
    
with open(config_path, "r") as config_fd:
    config = yaml.load(config_fd)
    
print(config)
#print(yaml.dump(config))

ordereddict([('samples', 'config/samples.tsv'), ('sequences', ordereddict([('gltB2', 'resources/gltB2.fa')])), ('conditions', ordereddict([('probe', ['1M7', 'NMIA', 'MIA', 'BzCN', 'DMS', 'CMCT']), ('temperature', ['37', '50']), ('magnesium', ['Mg', 'noMg'])])), ('allow_auto_import', True), ('format', ordereddict([('condition', '{id}_{probe}_{temperature}_{magnesium}_{interaction}'), ('control_condition', '{id}_{control}_of_{probe}_{temperature}_{magnesium}_{interaction}'), ('message', '{wildcards.rna_id} with {wildcards.probe} with {wildcards.temperature} with {wildcards.magnesium} with {wildcards.interaction}')])), ('rawdata', ordereddict([('path_prefix', '../../biodata'), ('type', 'fluo-ceq8000'), ('control', 'DMSO')])), ('probe', ordereddict([('1M7', ordereddict([('reactive_nucleotides', ['A', 'C', 'G', 'U'])])), ('BzCN', ordereddict([('reactive_nucleotides', ['A', 'C', 'G', 'U'])])), ('NMIA', ordereddict([('reactive_nucleotides', ['A', 'C', 'G', 'U'])])), ('NIA', ordereddict([('rea

In [36]:
def nested_dict_get(data: dict, position : []):
    try:
        subdata = data
        for pos in position:
            subdata = subdata[pos]
        
        return subdata
    except:
        return None

def nested_dict_set(data: dict, position : [], value):    
    subdata = data
    for pos in position[:-1]:
        subdata = subdata[pos]
    
    subdata[position[-1]] = value

class FileMV():
    def __init__(self, value, **kwargs):
        self._widget = FileChooser(**kwargs)
    
    @property
    def value(self):
        return self._filepicker.value
    
    @property
    def widget(self):
        return self._widget
    
class EnumMV():
    def __init__(self, value, listMV, **kwargs):
        val = value if value in listMV.keys else None
        self._widget = widgets.Dropdown(value=val, options=listMV.keys)
    
    @property
    def value(self):
        return self._widget.value
    
    @property
    def widget(self):
        return self._widget
    
class TextMV():
    def __init__(self, value=None, **kwargs):
        self._widget = widgets.Text(str(value) if value is not None else None, **kwargs)
    
    @property
    def value(self):
        return self._widget.value
    
    @property
    def widget(self):
        return self._widget

class EnumConditionMV():
    def __init__(self, value, conditionsMV, **kwargs):
        self._conds = {}
        if value is None:
            value = {}
        self._widget = widgets.VBox()
        for key, enumvalues in conditionsMV.value.items():
            try:
                val = str(value[key])
            except:
                val = None
            val = val if val in enumvalues else None
            self._conds[key] = widgets.Dropdown(value=val, options=enumvalues)
    @property
    def value(self):
        value = {}
        for key, val in self._conds.items():
            value[key] = val.value
        return value
    @property
    def widget(self):
        
        items = []
        for key, cw in self._conds.items():
            items.append(widgets.HBox([widgets.HTML(key), cw]))
            
        self._widget.children = items
        return self._widget
    
class IpanemapPoolMV():
    def __init__(self, value, conditionsMV, sequenceMV, **kwargs):
        self._widget = widgets.Box()
        if value is None:
            value = {"id": "", "rna_id": None, "conditions": {}, "alignements": {}}
        self._id = TextMV(value['id'])
        self._rna_id = EnumMV(value['rna_id'], sequenceMV)
        self._conditions = ListMV(value['conditions'], keyMV=EnumConditionMV, conditionsMV=conditionsMV)
        aln = {} if "alignements" not in value or value['alignements'] is None else value['alignements']
            
        self._alignements = DictMV(aln)
    
    @property
    def value(self):
        return {
            "id": self._id.value,
            "rna_id": self._rna_id.value,
            "conditions": self._conditions.value,
            "alignements": self._alignements.value
        }       
    
    @property
    def widget(self):
        self._widget.children = [widgets.VBox([
            self._id.widget,
            self._rna_id.widget,
            self._conditions.widget,
            self._alignements. widget
        ])]
        return self._widget

class DictMV():   
    def _notify():
        for w in notify_list:
            w.notify_change({"name":"external", "old":"", "new":""})
    def __init__(self, value : dict, intro: str = "", keyMV = TextMV, valueMV = TextMV, keyLabel="", valueLabel = "",**kwargs): 
        if value is None:
            value = {}
        self._keyLabel = keyLabel
        self._valueLabel = valueLabel
        self._keyMV = keyMV
        self._valueMV = valueMV
        self._kwargs = kwargs
        self._notify_list = []
        self._dic = {}
        for k, v in value.items():
            self._dic[uuid4()] = (keyMV(k, **kwargs), valueMV(v, **kwargs))
        self._root_container = widgets.Box()
        self._intro = intro
        
    def change_key(self, uid, change):
        key,value = self._dic[uid]
        self._dic[uid] = (self._keyMV(change['new'], **self._kwargs), value)
        
    def change_value(self, uid, change):
        key,value = self._dic[uid]
        self._dic[uid] = (key, self._valueMV(change['new'], **self._kwargs))
        
    def delete_item(self, uid, obj):
        del self._dic[uid]
        self.draw()

    def add_item(self, obj = None):
        key = self._keyMV(None, **self._kwargs)
        value = self._valueMV(None, **self._kwargs)
        self._dic[uuid4()] = (key, value)
        self.draw()

    def draw(self):
        items = [widgets.HBox([widgets.Label(""), widgets.Label(self._keyLabel), widgets.Label(self._valueLabel)])]
        for uid, (key, value) in self._dic.items():
            items.append(self.draw_item(uid,key,value))
        
        add_wg = widgets.Button(description="Add", icon="plus")
        add_wg.on_click(partial(self.add_item))
        self._root_container.children = (widgets.VBox([widgets.HTML(value=self._intro),widgets.VBox(items), add_wg]),)
        
    def draw_item(self, uid, key, value):
        rm_wg = widgets.Button(description="", icon="minus")
        rm_wg.on_click(partial(self.delete_item,  uid))
        key_wg = key.widget
        key_wg.observe(partial(self.change_key,uid),names="value")
        value_wg = value.widget
        value_wg.observe(partial(self.change_value,uid),names="value")
        item_wg = widgets.HBox([rm_wg, key_wg, value_wg])
        
        return item_wg

    @property
    def keys(self):
            return [el[0].value for el in self._dic.values()]
    @property 
    def value(self):
        return { el[0].value: el[1].value for el in self._dic.values()}
    
    @property
    def widget(self):
        self.draw()
        return self._root_container

    
class ListMV(DictMV):
    def __init__(self, value : list, intro: str = "", keyMV = TextMV, valueMV = TextMV, keyLabel="", valueLabel = "", **kwargs):
        if value is None:
            value = []
        self._keyLabel = keyLabel
        self._valueLabel = valueLabel
        self._keyMV = keyMV
        self._valueMV = valueMV
        self._kwargs = kwargs
        self._notify_list = []
        self._dic = {}
        for k in value:
            self._dic[uuid4()] = (keyMV(k, **kwargs), TextMV(""))
        self._root_container = widgets.Box()
        self._intro = intro    
    
    
    def draw_item(self, uid, key, value):
        rm_wg = widgets.Button(description="", icon="minus")
        rm_wg.on_click(partial(self.delete_item,  uid))
        key_wg = key.widget
        key_wg.observe(partial(self.change_key,uid),names="value")
        item_wg = widgets.HBox([rm_wg, key_wg])
        
        return item_wg
    
    @property
    def value(self):
        return self.keys

@dataclass
class ConfigViewer():
    config : dict
    
    def init_view(self):
        
        self._value_wgs = {}
        
        gen_list = self.init_general()
        sub_list = self.init_subsequence()
        self.init_seq_list()
        self.init_conditions()
        self.init_formatting()
        self.init_ipanemap()
        save = widgets.Button(description="Save")
        save.on_click(self.save)
        
        self.view = widgets.VBox(
            [gen_list,
             widgets.HTML("<h2>Split sequences </h2>"),
             sub_list,
             widgets.HTML("<h2>Sequences</h2>"),
             self._sequences_wgs.widget,
             widgets.HTML("<h2>Experimental conditions</h2>"),
             self._conditions_wgs.widget, 
             widgets.HTML("<h2>File Schema"),
             self._format_wgs,
             widgets.HTML("<h2>Ipanemap</h2>"),
             self._ipanemap_wgs.widget,
             save
            ])

        return self.view
    
    def init_seq_list(self):
        intro = ("The full sequence of the studied RNA in fasta format."
                "The fasta file should contain a description line containing the origin of the sequence"
                "(sequence database) with its unique identifier")
        self._sequences_wgs = DictMV(self.config["sequences"], intro=intro, valueMV=FileMV)
    
        return self._sequences_wgs
    
    def init_conditions(self):
        self._conditions_wgs = DictMV(self.config["conditions"], valueMV=ListMV)
        return self._conditions_wgs
    
    def min_cond_format_str(self):
        conds = self._conditions_wgs.value
        format_str = ""
        
        for cond in conds:
            if cond == "":
                continue
            cond_str = "{" + cond + "}"
            if cond_str in ["temp", "temperature"]:
 
                cond_str = "T{" + cond + "}C"
            format_str += "_" + cond_str
        return format_str        
    
    def gen_condition_format_str(self):
        return "{id}" + self.min_cond_format_str()
    
    def gen_control_condition_format_str(self):
        return "{id}_{control}_of" + self.min_cond_format_str()
    
    def gen_message_format_str(self):
        conds = self._conditions_wgs.value
        format_str = "{wildcards.rna_id}"
        
        for cond in conds:
            if cond == "":
                continue
            cond_str = "{wildcards." + cond + "}"
            if cond_str in ["temp", "temperature"]:
 
                cond_str = "T{wildcards." + cond + "}C"
            format_str += " with " + cond_str
        return format_str   
    
    def sync_formatting(self, obj):
        
        self._value_wgs[("format", "condition")].value = self.gen_condition_format_str()
        self._value_wgs[("format", "control_condition")].value = self.gen_control_condition_format_str()
        self._value_wgs[("format", "message")].value = self.gen_message_format_str()

        
    def init_formatting(self):
        syncbutton = widgets.Button(description="Synchronize name with conditions")
        syncbutton.on_click(self.sync_formatting)
        form = [widgets.HTML(value="File naming scheme"),
                self.add_value_widget(widgets.Text, ("format", "condition"), description="File naming schema"),
                self.add_value_widget(widgets.Text, ("format", "control_condition"), description="Control file naming schema"),
                self.add_value_widget(widgets.Text, ("format", "message"), descrption="Task message"),
                syncbutton
               ]
        self._format_wgs = widgets.VBox(form)
        return form

    def init_ipanemap(self):
        self._ipanemap_wgs = ListMV(self.config['ipanemap']['pools'],keyMV=IpanemapPoolMV, conditionsMV=self._conditions_wgs, sequenceMV = self._sequences_wgs)
        
        
    #def init_footprint(self):
        
    
    def add_value_widget(self, widgettype, pos : [], **kwargs):
        wg = widgettype(value=nested_dict_get(config, pos), **kwargs)
        self._value_wgs[pos] = wg
        return wg

        
    def init_general(self):
        gen = [
            widgets.HTML(value="<h1>Shape-CE pipeline configurator</h1><h2>General settings</h2>"),
            self.add_value_widget(widgets.Text, ("project_name",), description="Project Name")
        
        ]
        return widgets.VBox(gen)
    

    def init_subsequence(self):
        gen = [
        widgets.HTML(value=("When appling shape-ce method on large RNA, "
                            "you might need use several reverse transcription starting points, "
                            "in order to get reactivity along the entire sequence. If so you need to activate "
                            "«use sub-sequence»")),
        self.add_value_widget(widgets.Checkbox, ("qushape", "use_subsequence"), description="Use subsequence")
        ]
        return widgets.VBox(gen)
            
    
    def save(self, obj=None):
        for pos, mv in self._value_wgs.items():
            nested_dict_set(self.config, pos, mv.value)
        self.config["sequences"] = self._sequences_wgs.value
        self.config["conditions"] = self._conditions_wgs.value
        self.config["ipanemap"]["pools"] = self._ipanemap_wgs.value
        
        with open("test.yaml", "wb") as file:
            yaml.dump(self.config, file)
                
viewer = ConfigViewer(config)
viewer.init_view()
display(viewer.view)



In [7]:
#- General:
#    - Project name
#    
#    - subsequences
    
#- Sequences
#- Conditions
#  - With accepted values
#- Use subsequences
#- Ipanemap pools (all vs some)
#- footprint compares (all vs some)

